In [ ]:
# Dataset preparation
import pandas as pd
import json


df = pd.read_csv("hf://datasets/nbertagnolli/counsel-chat/20220401_counsel_chat.csv")
df.head()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256


In [8]:
# Drop rows with missing key values
df_clean = df.dropna(subset=["questionText", "answerText", "topic"])

# Calculate combined score (upvotes + views) and rank per questionText
df_clean['score'] = df_clean['upvotes'] + df_clean['views']
df_ranked = df_clean.sort_values(['questionText', 'score'], ascending=[True, False])

# Keep top n rows per questionText (e.g., n=1 for highest score)
n = 1  # Adjust n as needed (e.g., 2 for top 2)
df_filtered = df_ranked.groupby('questionText').head(n).reset_index(drop=True)

# Display the filtered DataFrame
print("Length of dataset before: ", len(df_clean))
print("Length of dataset after: ", len(df_filtered))
print("Filtered DataFrame with top scores per questionText:")
df_filtered.head()

Length of dataset before:  2612
Length of dataset after:  863
Filtered DataFrame with top scores per questionText:


<ipython-input-8-872c60ed48b5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['score'] = df_clean['upvotes'] + df_clean['views']


,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,score
0,353,I'm a 19 year old male with low self esteem. I...,A few nights ago I talked to this girl I know ...,https://counselchat.com/questions/i-m-a-19-yea...,self-esteem,Amy Fortney ParksChild & Adolescent Psychologi...,https://counselchat.com/therapists/amy-fortney...,Hey! It takes a lot of courage to share your ...,1,232,233
1,11,Why am I experiencing dfficulty maintaining an...,A few years ago I was making love to my wife w...,https://counselchat.com/questions/why-am-i-exp...,depression,"Pamela SuraciBuild on your strengths, grow in ...",https://counselchat.com/therapists/pamela-suraci,"If you haven't already, please see a doctor. ...",0,315,315
2,127,I'm scared to go to a doctor or take anti-depr...,A friend of mine taking psychology advised I g...,https://counselchat.com/questions/i-m-scared-t...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,I admire your courage for stating your view ab...,1,179,180
3,111,I'm depressed and unable to connect with peopl...,A girl and I were madly in love. We dated for ...,https://counselchat.com/questions/i-m-depresse...,depression,"Lynda MartensMarriage & Family Therapist, MSc,...",https://counselchat.com/therapists/ms-lynda-ma...,"Hi Boise, I'm sorry that you've lost this love...",2,125,127
4,193,How can I get over my fear of public embarrass...,"A lot of times, I avoid situations where I am ...",https://counselchat.com/questions/how-can-i-ge...,anxiety,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question. First,...",1,345,346


In [ ]:
# Create Q&A instruction-tuning dataset

# {"messages": [{"role": "system", "content": "You are a useful and harmless assistant"}, {"role": "user", "content": "Tell me tomorrow's weather"}, {"role": "assistant", "content": "Tomorrow's weather will be sunny"}]}
# {"messages": [{"role": "system", "content": "You are a useful and harmless math calculator"}, {"role": "user", "content": "What is 1 + 1?"}, {"role": "assistant", "content": "It equals 2"}, {"role": "user", "content": "What about adding 1?"}, {"role": "assistant", "content": "It equals 3"}]}
system_prompt = ""
with open("system_prompt.txt", "r", encoding="utf-8") as file:
    system_prompt = file.read().strip()
    print("Using system_prompt: \n", system_prompt)

qa_data = []
for _, row in df_filtered.iterrows():
    qa_data.append({
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": row["questionText"]},
            {"role": "assistant", "content": row["answerText"]}
        ]
        # "input": row["questionText"],
        # "output": row["answerText"]
    })

# Save Q&A fine-tuning data
with open("qa_train.jsonl", "w") as f:
    for item in qa_data:
        f.write(json.dumps(item) + "\n")
    print("Q&A dataset created.")

# Create classification dataset
clf_data = []
for _, row in df_filtered.iterrows():
    clf_data.append({
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": row["questionText"]},
            {"role": "assistant", "content": row["topic"]}
        ]
        # "input": row["questionText"],
        # "output": row["topic"]
    })

# Save classification fine-tuning data
with open("classification_train.jsonl", "w") as f:
    for item in clf_data:
        f.write(json.dumps(item) + "\n")
    print("Classification dataset created.")


Q&A dataset created.
Classification dataset created.
